# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
import pandas as pd
import numpy as np
import ssl
import matplotlib.pyplot as plt
import json

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [2]:
#la linea de abajo es para que mi compu pueda descargar el archivo sin tener problema con el certificado ssl
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.display.max_columns = None
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')
loans


/tmp/ipykernel_6117/637093913.py:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,NaN,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,Aug-1994,750.0,754.0,0.0,NaN,NaN,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.370000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,Jun-2016,12017.81,NaN,Nov-2017,684.0,680.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,149140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0,NaN,NaN,NaN,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,NaN,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,NaN,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,Sep-1989,710.0,714.0,2.0,42.0,NaN,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.690000,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,NaN,Feb-2017,564.0,560.0,0.0,59.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,162110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19400.0,NaN,NaN,NaN,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,42.0,1.0,42.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,

Buen uso del gzip para extraer el archivo sin error.

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [3]:
column_types = loans.dtypes
with pd.option_context('display.max_rows', None):
    print(column_types)


Unnamed: 0                                      int64
id                                              int64
member_id                                     float64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                  

Considero que es una buena idea usar el print en lugar del metodo de deafult de Jupyter para mostrar las columnas, ya que de esta manera se tiene mas control sobre como se ve y que se ve.

In [4]:
'''
last_pymnt_d -> date
hardship_start_date -> date                           
hardship_end_date  -> date                             
payment_plan_start_date  -> date
settlement_date -> date

'''
with pd.option_context('display.max_rows', 5):
    print(loans['payment_plan_start_date'])


0        NaN
1        NaN
        ... 
99998    NaN
99999    NaN
Name: payment_plan_start_date, Length: 100000, dtype: object


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [5]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [6]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


Bien hecho!

### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [7]:
# COdigo guardar
datos_dict.to_pickle('datos_dict.pkl')

In [8]:
# Codigo para cargar
datos_dict_pickle = pd.read_pickle('datos_dict.pkl')
datos_dict_pickle

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


Bien hecho!

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [9]:
# Manejos de tipos 1
''' 
Las siguientes variables almacenan datos de tipo object que 
pueden ser almacenados en tipo date para poder analizarlos posteriormente
last_pymnt_d -> date
hardship_start_date -> date                           
hardship_end_date  -> date                             
payment_plan_start_date  -> date
settlement_date -> date
'''
# Tu codigo aqui
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'], format='%b-%Y')
loans['hardship_start_date'] = pd.to_datetime(loans['hardship_start_date'], format='%b-%Y')
loans['hardship_end_date'] = pd.to_datetime(loans['hardship_end_date'], format='%b-%Y')
loans['payment_plan_start_date'] = pd.to_datetime(loans['payment_plan_start_date'], format='%b-%Y')
loans['settlement_date'] = pd.to_datetime(loans['settlement_date'], format='%b-%Y')



Buen manejo de columnas que contienen una fecha.

In [10]:
# Manejos de tipos 2
# Tu codigo aqui


In [11]:
column_types = loans.dtypes
column_types

Unnamed: 0                        int64
id                                int64
member_id                       float64
loan_amnt                       float64
funded_amnt                     float64
                              ...      
settlement_status                object
settlement_date          datetime64[ns]
settlement_amount               float64
settlement_percentage           float64
settlement_term                 float64
Length: 151, dtype: object

Considero que hay otras columnas que pudieron beneficiarse de un tratamiento similar.

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [12]:
#Primero verificamos que columnas tienen gran número de Nans
#en este caso los que sean más de 10000
suma_nans = loans.isna().sum()
columnas_selecc = suma_nans.between(10000, 90000)
columnas_con_nans = loans.loc[:, columnas_selecc]
columnas_sin_datos = loans.loc[:, suma_nans > 90000]
print(columnas_con_nans.columns)
print(columnas_sin_datos.columns)

Index(['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d',
       'mths_since_last_major_derog', 'mths_since_recent_bc_dlq',
       'mths_since_recent_revol_delinq'],
      dtype='object')
Index(['member_id', 'desc', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'open_acc_6m', 'open_act_il',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'inq_fi', 'total_cu_tl', 'inq_last_12m', 'revol_bal_joint',
       'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_t

Buena manera de ver cuales columnas requieren de mayor tratamiento con los nans o si conviene hacerlo.

In [13]:
#Luego verificamos si la distribución es normal o sesgada
columnas_con_nans.hist(figsize=(100, 100))
plt.show()

Las graficas estan bien pero quizas seria mejor aumentar el tamaño de las titulos y de los ejes para que fueran mucho mas legibles.

Podemos ver que la variable columnas_sin_datos contiene más de 90 mil Nans, por lo que se eliminarán 

Adicionalmente la variable columnas_con_nans almacena las columnas a las que se les aplicará una estrategia de imputación de mediana puesto que la distribución es sesgada
        

In [14]:
# Tu codigo aqui
imputacion = loans.drop(columns=columnas_sin_datos.columns)
imputacion = imputacion.fillna(imputacion.median())

/tmp/ipykernel_6117/4288554406.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  imputacion = imputacion.fillna(imputacion.median())
/tmp/ipykernel_6117/4288554406.py:3: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  imputacion = imputacion.fillna(imputacion.median())


No todas las columnas aqui son numericas por lo que usar imputacion por medianas no es una buena idea, aquellas que tienen fechas, string o algun otro objeto deben de ser tratados diferentes quizas reemplazando los datos por cosas como "Missing" o similares.

In [15]:
# TUcodigo aqui

### Codigo para salvar y cargar JSONs

In [16]:
llaves = list(loans.columns)
valores = [None] * len(llaves)
zipped = zip(llaves, valores)
dict_estrategia = {k: v for k, v in zipped}
lista_columnas_con_nans = list(columnas_con_nans.columns)
lista_columnas_sin_datos =list(columnas_sin_datos.columns)
for columna in lista_columnas_con_nans:
    dict_estrategia[columna] = {"estrtegia": "Imputación de mediana", "valor": "mediana"}

for columna in lista_columnas_sin_datos:
    dict_estrategia[columna] = {"estrategia": "Eliminación por falta de datos", "valor":"Ninguno"}

with open("datos.json", "w") as archivo:
    json.dump(dict_estrategia, archivo)
print(dict_estrategia)

with open("datos.json", "r") as archivo:
    # Lee los datos del archivo usando json.load()
    datos_json = json.load(archivo)

{'Unnamed: 0': None, 'id': None, 'member_id': {'estrategia': 'Eliminación por falta de datos', 'valor': 'Ninguno'}, 'loan_amnt': None, 'funded_amnt': None, 'funded_amnt_inv': None, 'term': None, 'int_rate': None, 'installment': None, 'grade': None, 'sub_grade': None, 'emp_title': None, 'emp_length': None, 'home_ownership': None, 'annual_inc': None, 'verification_status': None, 'issue_d': None, 'loan_status': None, 'pymnt_plan': None, 'desc': {'estrategia': 'Eliminación por falta de datos', 'valor': 'Ninguno'}, 'purpose': None, 'title': None, 'zip_code': None, 'addr_state': None, 'dti': None, 'delinq_2yrs': None, 'earliest_cr_line': None, 'fico_range_low': None, 'fico_range_high': None, 'inq_last_6mths': None, 'mths_since_last_delinq': {'estrtegia': 'Imputación de mediana', 'valor': 'mediana'}, 'mths_since_last_record': {'estrtegia': 'Imputación de mediana', 'valor': 'mediana'}, 'open_acc': None, 'pub_rec': None, 'revol_bal': None, 'revol_util': None, 'total_acc': None, 'initial_list_st

Buen uso de los JSON, quizas agregarle una etiqueta mas alla de none a las columnas que no se imputaron, y de nuevo, algunas columnas que fueron tratadas con mediana son string o fechas por lo que no creo que esten bien descritas.